In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt

import torch
print(torch.__version__)
import torchvision
from torchvision import datasets, transforms
import torch.nn as nn

from IPython.display import clear_output

import helper

0.4.1


# Batch Gradient Descent and Binary Classification

In [30]:
data_dir = '/Users/akshaykulkarni/flowers/train/'
batch_size = 1024
tf = transforms.Compose([transforms.Resize((255, 255)),
                         transforms.ToTensor(),
                        ])

train_dataset = datasets.ImageFolder(data_dir, transform = tf)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True, drop_last = True)

data_dir = '/Users/akshaykulkarni/flowers/test/'

test_dataset = datasets.ImageFolder(data_dir, transform = tf)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = 32, shuffle = True, drop_last = True)

In [51]:
# Loading all the images in the single batch and flattening
images, labels = next(iter(train_loader))
print(images.shape)
images = images.view(-1, images.shape[0]).float()
print(images.shape)
images /= 255
labels = labels.float()
print(labels.shape)

torch.Size([1024, 3, 255, 255])
torch.Size([195075, 1024])
torch.Size([1024])


### Initializing the parameters using no. of nodes and no. of layers

In [91]:
# no. of layers
layers = 3
# no. of nodes in each layer in a list including input and output layers
num_nodes = [images.shape[0], 512, 1]
num_epochs = 5

# initializing weights according to no. of layers and no. of nodes
assert(layers == len(num_nodes))
W = {}
b = {}
Z = {}
dw = {}
db = {}
da = {}
dz = {}
for i in range(1, layers):
    var_name = str(i)
    W[var_name] = torch.empty(num_nodes[i - 1], num_nodes[i])
    # Xavier initialization of weights
    nn.init.xavier_uniform_(W[var_name], gain=nn.init.calculate_gain('sigmoid'))
    b[var_name] = torch.randn(num_nodes[i], 1)

### Loss and Activation Functions

In [92]:
def binary_cross_entropy(A, Y):
    loss = (-1 / batch_size) * (torch.matmul(torch.log(A), Y) + torch.matmul(torch.log(1 - A), (1 - Y)))
    return loss

def sigmoid(x):
    return (1 / (1 + torch.exp(-x)))

### Forward Propagation

In [93]:
def forward(x, y):
    out = x
    zero = str(0)
    A[zero] = x
    for i in range(1, layers):
        var_name = str(i)
        out = torch.add(torch.matmul(W[var_name].t(), out), b[var_name])
        Z[var_name] = out
        out = sigmoid(out)
        print(out.shape)
    
    loss = binary_cross_entropy(out, y)
    return loss, out

In [94]:
# loss, out = forward(images, labels)
# print(out)
# print(loss)

### Backward Propagation

In [109]:
def backward(A, Y, X):
    # for last layer
    Y = Y.view(1, Y.shape[0])
    last_layer = str(layers - 1)
    second_last_layer = str(layers - 2)
    dz[last_layer] = A - Y
    dw[last_layer] = (1 / batch_size) * torch.matmul(dz[last_layer], sigmoid(Z[second_last_layer]).t())
    dw[last_layer] = dw[last_layer].t()
    db[last_layer] = (1 / batch_size) * torch.sum(dz[last_layer], 1, keepdim = True)
    da[second_last_layer] = torch.matmul(W[last_layer], dz[last_layer])
    for i in range(layers - 2, 0, -1):
        var_name = str(i)
        prev_name = str(i - 1)
        g_dash_z = sigmoid(Z[var_name]) * (1 - sigmoid(Z[var_name]))
        dz[var_name] = da[var_name] * g_dash_z
        if i == 1:
            dw[var_name] = (1 / batch_size) * torch.matmul(dz[var_name], X.t())
        else:
            dw[var_name] = (1 / batch_size) * torch.matmul(dz[var_name], sigmoid(Z[prev_name]).t())
        dw[var_name] = dw[var_name].t()
        db[var_name] = (1 / batch_size) * torch.sum(dz[var_name], 1, keepdim = True)
        da[prev_name] = torch.matmul(W[var_name], dz[var_name])

In [110]:
def update_weights(learning_rate):
    for i in range(1, layers):
        var_name = str(i)
        print(W[var_name].shape)
        print(dw[var_name].shape)
        assert(W[var_name].shape == dw[var_name].shape)
        assert(b[var_name].shape == db[var_name].shape)
        W[var_name] -= learning_rate * dw[var_name]
        b[var_name] -= learning_rate * db[var_name]

### Training Loop

In [111]:
loss_list = list()
for j in range(num_epochs):
    if torch.cuda.is_available():
        for q in range(1, layers):
            var_name = str(q)
            W[var_name] = W[var_name].cuda()
            b[var_name] = b[var_name].cuda()
        
        images, labels = images.cuda(), labels.cuda()
    
    loss, pred = forward(images, labels)
    
    loss_list.append(loss.item())
    
    backward(pred, labels, images)
    
    update_weights(learning_rate = 0.01)
    
    print(f'Epoch : [{j} / {num_epochs} | Training Loss : {loss.item()}]')

torch.Size([512, 1024])
torch.Size([1, 1024])
torch.Size([195075, 512])
torch.Size([195075, 512])
torch.Size([512, 1])
torch.Size([512, 1])
Epoch : [0 / 5 | Training Loss : 0.701448917388916]
torch.Size([512, 1024])
torch.Size([1, 1024])
torch.Size([195075, 512])
torch.Size([195075, 512])
torch.Size([512, 1])
torch.Size([512, 1])
Epoch : [1 / 5 | Training Loss : 0.6963069438934326]
torch.Size([512, 1024])
torch.Size([1, 1024])
torch.Size([195075, 512])
torch.Size([195075, 512])
torch.Size([512, 1])
torch.Size([512, 1])
Epoch : [2 / 5 | Training Loss : 0.6942684650421143]
torch.Size([512, 1024])
torch.Size([1, 1024])
torch.Size([195075, 512])
torch.Size([195075, 512])
torch.Size([512, 1])
torch.Size([512, 1])
Epoch : [3 / 5 | Training Loss : 0.693467378616333]
torch.Size([512, 1024])
torch.Size([1, 1024])
torch.Size([195075, 512])
torch.Size([195075, 512])
torch.Size([512, 1])
torch.Size([512, 1])
Epoch : [4 / 5 | Training Loss : 0.6931537985801697]
